# Criando o grafo de analise do wikipidea

O objetivo desta atividade é criar um grafo em que os nós representem páginas da Wikipedia, utilizando os links presentes nessas páginas para estabelecer conexões entre os nós.

a partir do notebook [Case Study: constructing a network of wikipedia pages](https://github.com/ivanovitchm/datastructure/blob/main/lessons/week_09/Wikipedia.ipynb) desenvolvido pelo professor Ivanovitch foi possível criar o grafo.

## importando a lib do wikipédia

In [1]:
!pip install wikipedia

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=8360319cf4e30ea86539f50a552b382126b69e67b2ab024a760cd5af7b57854c
  Stored in directory: /home/eduardo09/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [3]:
from operator import itemgetter
import networkx as nx
import wikipedia
import matplotlib.pyplot as plt

In [8]:
#https://en.wikipedia.org/wiki/Complex_network
SEED1 = "Olympic Games".title()
SEED2 = "Paris".title()
SEED3 = "World War".title()

STOPS = ("International Standard Serial Number",
         "International Standard Book Number",
         "National Diet Library",
         "International Standard Name Identifier",
         "International Standard Book Number (Identifier)",
         "Pubmed Identifier",
         "Pubmed Central",
         "Digital Object Identifier",
         "Arxiv",
         "Proc Natl Acad Sci Usa",
         "Bibcode",
         "Library Of Congress Control Number",
         "Jstor",
         "Doi (Identifier)",
         "Isbn (Identifier)",
         "Pmid (Identifier)",
         "Arxiv (Identifier)",
         "Bibcode (Identifier)")

In [9]:
todo_lst = [(0, SEED)] # The SEED is in the layer 0
todo_set = set(SEED) # The SEED itself
done_set = set() # Nothing is done yet

In [10]:
g = nx.DiGraph()
layer, page = todo_lst[0]

In [11]:
%%time
while layer < 2:
  # Remove the name page of the current page from the todo_lst,
  # and add it to the set of processed pages.
  # If the script encounters this page again, it will skip over it.
  del todo_lst[0]
  done_set.add(page)

  # Show progress
  print(layer, page)

  # Attempt to download the selected page.
  try:
    wiki = wikipedia.page(page)
  except:
    print("Could not load", page)
    layer, page = todo_lst[0]
    continue

  for link in wiki.links:
    link = link.title()
    if link not in STOPS and not link.startswith("List Of"):
      if link not in todo_set and link not in done_set:
        todo_lst.append((layer + 1, link))
        todo_set.add(link)
      g.add_edge(page, link)
  layer, page = todo_lst[0]

0 Olympic Games
1 117Th Ioc Session
1 1500 Metres
1 16-Inch Softball
1 1896 Summer Olympics
Could not load 1896 Summer Olympics
1 1900 Summer Olympics
Could not load 1900 Summer Olympics
1 1904 Summer Olympics
Could not load 1904 Summer Olympics
1 1906 Intercalated Games
1 1908 Summer Olympics
1 1912 Summer Olympics
1 1916 Summer Olympics
1 1920 Summer Olympics
Could not load 1920 Summer Olympics
1 1924 Summer Olympics
1 1924 Winter Olympics
1 1928 Summer Olympics
1 1928 Winter Olympics
1 1930 British Empire Games
1 1932 Summer Olympics
1 1932 Winter Olympics
1 1936 Summer Olympics
Could not load 1936 Summer Olympics
1 1936 Winter Olympics
1 1940 Summer Olympics
1 1940 Winter Olympics
1 1944 Summer Olympics
Could not load 1944 Summer Olympics
1 1944 Winter Olympics
Could not load 1944 Winter Olympics
1 1948 Summer Olympics
Could not load 1948 Summer Olympics
1 1948 Winter Olympics
1 1952 Summer Olympics
1 1952 Winter Olympics
Could not load 1952 Winter Olympics
1 1956 Summer Olympics
1

In [12]:
print("{} nodes, {} edges".format(len(g), nx.number_of_edges(g)))

243967 nodes, 809104 edges
